# 07. 시간 기반 라벨링

## 목적
- 폐업까지 남은 개월 수 계산
- 5단계 상태 라벨 생성
- 조기 경보 시점 분석

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
df = pd.read_csv(data_path)

print(f"데이터: {len(df):,}개")

데이터: 86,263개


## 날짜 변환 및 months_to_close 계산

In [2]:
df['기준년월'] = pd.to_datetime(df['기준년월'].astype(str), format='%Y%m')
df['폐업일'] = df['폐업일'].apply(lambda x: pd.to_datetime(str(int(x)), format='%Y%m%d', errors='coerce') if pd.notna(x) else pd.NaT)

def calculate_months_to_close(row):
    if pd.isna(row['폐업일']):
        return None
    months_diff = (row['폐업일'].year - row['기준년월'].year) * 12 + (row['폐업일'].month - row['기준년월'].month)
    return months_diff

df['months_to_close'] = df.apply(calculate_months_to_close, axis=1)

print("\n폐업 가맹점 months_to_close 통계:")
print(df[df['months_to_close'].notna()]['months_to_close'].describe())


폐업 가맹점 months_to_close 통계:
count    2334.000000
mean       16.006427
std         7.757051
min         0.000000
25%        10.000000
50%        16.000000
75%        22.000000
max        31.000000
Name: months_to_close, dtype: float64


## 5단계 라벨링

In [3]:
def label_status(months):
    if pd.isna(months):
        return '정상영업'
    elif months < 0:
        return '이미폐업'
    elif months <= 3:
        return 'D-3m'
    elif months <= 6:
        return 'D-6m'
    elif months <= 9:
        return 'D-9m'
    elif months <= 12:
        return 'D-12m'
    else:
        return '정상영업'

df['상태'] = df['months_to_close'].apply(label_status)

print("\n상태별 분포:")
print(df['상태'].value_counts().sort_index())


상태별 분포:
상태
D-12m      321
D-3m       126
D-6m        96
D-9m       316
정상영업     85404
Name: count, dtype: int64


## 상태별 평균 지표 계산

### 의미
- D-3m: 3개월 내 폐업 예상 → 긴급 개입
- D-6m~12m: 조기 경보 단계
- 시간 기반 정책 수립 가능